
Multi-criteria decision making (MCDM) using Topsis

In [1]:
# Installing topsis pelo terminal
#pip install pymcdm

# import nbformat to handle notebook files to call AHP notebook with %run ../AHP/AHP.ipynb
# pip install nbformat

In [2]:
# Imports
# Executar o notebook AHP para carregar os pesos
%run ../../AHP/test/AHP.ipynb

# Executar o notebook read_csv para importar o dataframe para teste;
# Posteriormente será importada a classe responsável usada no projecto
# %run read_csv.ipynb

%run matrix_for_TOPSIS.ipynb
mydata # <- Dataframe importado com os dados
print(mydata)


# Estrutura do DataFrame necessária para o funcionamento do TOPSIS:
# |─────────────|───────────|─────────────────|───────────────────|─────────────────|────────────|─────────────────────────|
# │ Donor_id    │ HLA Match │ CMV Serostatus  │  Donor Age Group  │ Gender Match    │ ABO Match  │ Expected Survival Time  │
# ├─────────────┼───────────┼─────────────────┼───────────────────┼─────────────────┼────────────┼─────────────────────────┤
# │ str/object  │ int       │ int             │ int               │ int             │ int        │ int                     │
# └─────────────┴───────────┴─────────────────┴───────────────────┴─────────────────┴────────────┴─────────────────────────┘


import pandas as pd
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.normalizations import vector_normalization

Pesos AHP:
{'HLA': np.float64(0.4029), 'Idade_dador': np.float64(0.3088), 'CMV': np.float64(0.1423), 'ABO': np.float64(0.0604), 'Sexo': np.float64(0.0555), 'Survival_Time': np.float64(0.0302)}

Consistency Ratio (CR):
0.0277
    Donor_id  HLA Match  CMV Serostatus  Donor Age Group  Gender Match  \
0          1          7               0                0             0   
1          2          8               1                1             1   
2          3          9               2                2             0   
3          4          7               3                0             1   
4          5         10               0                2             0   
5          6         10               3                0             1   
6          7          8               0                1             1   
7          8          9               1                0             0   
8          9          7               2                2             1   
9         10         10            

In [3]:
# Drop the first column (ids) and convert to NumPy array
mydata_no_first_column = mydata.iloc[:, 1:].values.astype(int)
mydata_no_first_column

array([[   7,    0,    0,    0,    0,  100],
       [   8,    1,    1,    1,    1,  200],
       [   9,    2,    2,    0,    0,  300],
       [   7,    3,    0,    1,    0,   50],
       [  10,    0,    2,    0,    1,  500],
       [  10,    3,    0,    1,    0,   40],
       [   8,    0,    1,    1,    0,  220],
       [   9,    1,    0,    0,    1,  340],
       [   7,    2,    2,    1,    0,   90],
       [  10,    0,    1,    0,    1,  480],
       [   9,    3,    0,    1,    1,  310],
       [   8,    1,    2,    0,    0,  180],
       [  10,    2,    0,    1,    1, 1020],
       [   7,    0,    1,    0,    0,  110],
       [   9,    1,    1,    1,    0,  290],
       [   8,    3,    2,    0,    1,  160],
       [  10,    0,    0,    1,    0,  550],
       [   7,    2,    1,    0,    1,  130],
       [   9,    0,    2,    1,    1,  260],
       [   8,    1,    0,    0,    0,  210],
       [  10,    3,    1,    1,    1,  470],
       [   7,    1,    2,    0,    0,   95],
       [  

In [4]:
# Assigning the impacts
# 1 for maximization and -1 for minimization
# Maximize: HLA_match, donor_age_group, ABO_match, expected_time_survival, Gender_match
# Minimize: CMV_status

i = np.array([1, -1, 1, 1, 1, 1], dtype=int)
i

array([ 1, -1,  1,  1,  1,  1])

In [5]:
# Assigning the weights
# # Os pesos foram definidos usando o método de apoio à decisão multicritério AHP (Analytic Hierarchy Process)


# Os pesos abaixo são importados do notebook AHP
w = np.array([peso_HLA, peso_CMV, peso_Idade_dador, peso_Sexo, peso_ABO, peso_Survival_Time], dtype=float)
w

array([0.4029, 0.1423, 0.3088, 0.0555, 0.0604, 0.0302])

In [6]:
# Calling the topsis function
# scores = topsis(
#     data,        # matriz de decisão (alternativas × critérios)
#     weights,     # pesos dos critérios
#     impacts      # impactos: '1' ou '-1'
# )

topsis = TOPSIS(normalization_function=vector_normalization)
resultados = topsis(mydata_no_first_column, w, i)
resultados

array([0.32315432, 0.53186839, 0.69809175, 0.11725624, 0.86604141,
       0.22120216, 0.56062092, 0.29525171, 0.66877595, 0.59941338,
       0.21388612, 0.73643384, 0.29616144, 0.52728798, 0.53083326,
       0.6331733 , 0.37703876, 0.44132825, 0.87672629, 0.25073789,
       0.46443377, 0.70555446, 0.21989209, 0.58009138, 0.81431476])

In [7]:
# Cria tabela final

# Get the first column
primeira_coluna = mydata.iloc[:, 0]  # Keep as Series

# Convert results to Series
resultados_series = pd.Series(resultados, name='TOPSIS Score')

# Concatenate along columns
df_TOPSIS = pd.concat([primeira_coluna, resultados_series], axis=1)
print(df_TOPSIS)

# Sort by TOPSIS_Score in descending order (highest to lowest)
df_TOPSIS = df_TOPSIS.sort_values(by='TOPSIS Score', ascending=False)
df_TOPSIS.rename(columns={'TOPSIS Score': 'TOPSIS Rank'}, inplace=True)

df_TOPSIS # <- Tabela a ser retornada

    Donor_id  TOPSIS Score
0          1      0.323154
1          2      0.531868
2          3      0.698092
3          4      0.117256
4          5      0.866041
5          6      0.221202
6          7      0.560621
7          8      0.295252
8          9      0.668776
9         10      0.599413
10        11      0.213886
11        12      0.736434
12        13      0.296161
13        14      0.527288
14        15      0.530833
15        16      0.633173
16        17      0.377039
17        18      0.441328
18        19      0.876726
19        20      0.250738
20        21      0.464434
21        22      0.705554
22        23      0.219892
23        24      0.580091
24        25      0.814315


,Donor_id,TOPSIS Rank
18,19,0.876726
4,5,0.866041
24,25,0.814315
11,12,0.736434
21,22,0.705554
2,3,0.698092
8,9,0.668776
15,16,0.633173
9,10,0.599413
23,24,0.580091
